In [1]:
import json
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage,AIMessage

In [2]:
llm = init_chat_model(
    model="gemini-2.5-flash-lite",
    model_provider="google_genai",
    temperature=0.2,
    api_key=os.getenv("GOOGLE_GEMINI_API_KEY"),
)

In [3]:
DATASET_PROMPT = """
You are generating test cases for evaluating a prompt.

Generate a JSON array of objects.
Each object must have a "task", a "format" & a "solution_criteria" field

The task should ask for ONE of the following outputs:
- Python code
- JSON configuration
- A regular expression

The format field should include the type of task i.e. it is python/json/regex

The solution_criteria field should include some criterias to solve the task, this should be in just one line.

Do NOT include explanations.
Return ONLY valid JSON.
Generate exactly 3 tasks.
"""

In [4]:
def generate_dataset():
    messages = [HumanMessage(content=DATASET_PROMPT), AIMessage(content="```json")]

    response = llm.invoke(messages, stop=["```"])

    raw_json = response.content.strip()
    return json.loads(raw_json)

In [ ]:
def save_dataset(dataset, path="dataset.json"):
    with open(path, "w") as f:
        json.dump(dataset, f, indent=2)

In [6]:
if __name__ == "__main__":
    dataset = generate_dataset()
    print("Generated dataset: ")
    print(dataset)

    save_dataset(dataset, "tasks.json")
    print("Dataset saved.")

Generated dataset: 
[{'task': 'Write a Python function that takes a list of integers and returns a new list containing only the even numbers.', 'format': 'python', 'solution_criteria': 'The function should accept a list of integers and return a new list with only the even numbers from the input list.'}, {'task': "Create a JSON configuration for a simple web server that listens on port 8080 and serves files from a '/var/www/html' directory.", 'format': 'json', 'solution_criteria': "The JSON object should define 'port' with value 8080 and 'document_root' with value '/var/www/html'."}, {'task': 'Develop a regular expression to validate an email address.', 'format': 'regex', 'solution_criteria': "The regex should match a standard email format including a username, '@' symbol, domain name, and a top-level domain."}]
Dataset saved.
